In [ ]:
# %% [markdown]
# # FASE 2: Preprocesamiento y Limpieza de Texto
# ## Persona B - Procesamiento NLP

# %%
# Instalación de dependencias
# !pip install nltk pandas matplotlib seaborn

# %%
import sys
import os
sys.path.append('../scripts')

# %%
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from preprocessing import TextPreprocessor

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# %%
# Cargar datos crudos
print("Cargando datos crudos...")
df_raw = pd.read_csv('../data/raw/reviews_amazon_raw.csv')
print(f"Dataset cargado: {len(df_raw)} reseñas")
print(f"Columnas: {list(df_raw.columns)}")

# %%
# Mostrar información básica del dataset
print("INFORMACIÓN DEL DATASET CRUDO")
print("-" * 40)
print(f"Total de reseñas: {len(df_raw)}")
print(f"Reseñas con puntuación: {df_raw['puntuacion'].notna().sum()}")
print(f"Reseñas sin puntuación: {df_raw['puntuacion'].isna().sum()}")

# Distribución de puntuaciones
if df_raw['puntuacion'].notna().any():
    print("\nDistribución de puntuaciones:")
    print(df_raw['puntuacion'].value_counts().sort_index())

# %%
# Ejemplo de textos antes del procesamiento
print("\nEJEMPLOS DE TEXTO ANTES DEL PROCESAMIENTO:")
print("=" * 80)
for i in range(3):
    print(f"\nReseña {i+1} (Puntuación: {df_raw.loc[i, 'puntuacion']}):")
    print(f"Texto: {df_raw.loc[i, 'texto'][:150]}...")
    print("-" * 80)

# %%
# Inicializar preprocesador
print("\nInicializando preprocesador de texto...")
preprocessor = TextPreprocessor(language='spanish')

# %%
# Procesar una reseña de ejemplo para ver el pipeline
print("\nPROCESAMIENTO DE UNA RESEÑA DE EJEMPLO:")
print("=" * 80)

sample_text = df_raw.loc[0, 'texto']
print(f"Texto original:\n{sample_text}\n")
print("-" * 80)

result = preprocessor.preprocess_pipeline(sample_text, stem=True)

print(f"Texto limpio:\n{result['texto_limpio']}\n")
print(f"Texto sin stopwords:\n{result['texto_sin_stopwords']}\n")
print(f"Texto con stemming:\n{result['texto_stemmed']}\n")
print(f"Tokens sin stopwords: {result['tokens_sin_stopwords'][:10]}...")
print(f"Número de palabras original: {result['num_palabras_original']}")
print(f"Número de palabras sin stopwords: {result['num_palabras_sin_stopwords']}")

# %%
# Procesar todo el dataset
print("\n" + "=" * 80)
print("PROCESANDO TODO EL DATASET")
print("=" * 80)

df_processed = preprocessor.process_dataframe(df_raw, text_column='texto')

# %%
# Calcular métricas adicionales
df_processed = preprocessor.calculate_text_metrics(df_processed)

# %%
# Guardar dataset procesado
output_path = '../data/processed/reviews_preprocessed.csv'
df_processed.to_csv(output_path, index=False, encoding='utf-8')
print(f"\nDataset procesado guardado en: {output_path}")

# %%
# Análisis de las métricas calculadas
print("\nANÁLISIS DE MÉTRICAS DE TEXTO")
print("=" * 40)

# Estadísticas de longitud
print("\nLongitud de textos:")
print(f"  Original - Promedio: {df_processed['longitud_texto_original'].mean():.0f} chars")
print(f"  Original - Máximo: {df_processed['longitud_texto_original'].max():.0f} chars")
print(f"  Original - Mínimo: {df_processed['longitud_texto_original'].min():.0f} chars")
print(f"  Limpio - Promedio: {df_processed['longitud_texto_limpio'].mean():.0f} chars")

print("\nNúmero de palabras:")
print(f"  Original - Promedio: {df_processed['num_palabras_original'].mean():.0f} palabras")
print(f"  Limpio - Promedio: {df_processed['num_palabras_limpio'].mean():.0f} palabras")
print(f"  Reducción por stopwords: {((df_processed['num_palabras_original'] - df_processed['num_palabras_sin_stopwords']).mean()):.1f} palabras/reseña")

print(f"\nDensidad léxica promedio: {df_processed['densidad_lexica'].mean():.2%}")

# %%
# Visualización de métricas
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 1. Distribución de longitud de textos originales
axes[0, 0].hist(df_processed['longitud_texto_original'], bins=30, alpha=0.7, color='skyblue')
axes[0, 0].set_title('Distribución Longitud Textos Originales')
axes[0, 0].set_xlabel('Caracteres')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].grid(True, alpha=0.3)

# 2. Distribución de longitud de textos limpios
axes[0, 1].hist(df_processed['longitud_texto_limpio'], bins=30, alpha=0.7, color='lightgreen')
axes[0, 1].set_title('Distribución Longitud Textos Limpios')
axes[0, 1].set_xlabel('Caracteres')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].grid(True, alpha=0.3)

# 3. Distribución de número de palabras
axes[0, 2].hist(df_processed['num_palabras_original'], bins=30, alpha=0.5, label='Original', color='skyblue')
axes[0, 2].hist(df_processed['num_palabras_sin_stopwords'], bins=30, alpha=0.5, label='Sin stopwords', color='lightcoral')
axes[0, 2].set_title('Distribución Número de Palabras')
axes[0, 2].set_xlabel('Número de palabras')
axes[0, 2].set_ylabel('Frecuencia')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# 4. Densidad léxica por puntuación
if 'puntuacion' in df_processed.columns and df_processed['puntuacion'].notna().any():
    box_data = []
    labels = []
    for score in sorted(df_processed['puntuacion'].dropna().unique()):
        mask = df_processed['puntuacion'] == score
        box_data.append(df_processed.loc[mask, 'densidad_lexica'].dropna())
        labels.append(f'{score} estrellas')
    
    axes[1, 0].boxplot(box_data, labels=labels)
    axes[1, 0].set_title('Densidad Léxica por Puntuación')
    axes[1, 0].set_ylabel('Densidad Léxica')
    axes[1, 0].grid(True, alpha=0.3)

# 5. Relación entre longitud y puntuación
if 'puntuacion' in df_processed.columns and df_processed['puntuacion'].notna().any():
    scatter_data = df_processed.dropna(subset=['puntuacion'])
    axes[1, 1].scatter(scatter_data['puntuacion'], scatter_data['num_palabras_original'], 
                       alpha=0.6, color='purple')
    axes[1, 1].set_title('Longitud vs Puntuación')
    axes[1, 1].set_xlabel('Puntuación (estrellas)')
    axes[1, 1].set_ylabel('Número de palabras')
    axes[1, 1].grid(True, alpha=0.3)

# 6. Distribución de densidad léxica
axes[1, 2].hist(df_processed['densidad_lexica'], bins=30, alpha=0.7, color='gold')
axes[1, 2].set_title('Distribución Densidad Léxica')
axes[1, 2].set_xlabel('Densidad Léxica')
axes[1, 2].set_ylabel('Frecuencia')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../visualizations/metricas_preprocesamiento.png', dpi=150, bbox_inches='tight')
plt.show()

# %%
# Análisis de stopwords eliminadas
print("\nANÁLISIS DE STOPWORDS")
print("=" * 40)

# Contar frecuencia de palabras antes y después de eliminar stopwords
from collections import Counter

# Combinar todos los textos
all_tokens_original = []
all_tokens_clean = []

for idx, row in df_processed.iterrows():
    if isinstance(row['tokens'], list):
        all_tokens_original.extend(row['tokens'])
    if isinstance(row['tokens_sin_stopwords'], list):
        all_tokens_clean.extend(row['tokens_sin_stopwords'])

# Contar frecuencias
freq_original = Counter(all_tokens_original)
freq_clean = Counter(all_tokens_clean)

print(f"\nTotal tokens originales: {len(all_tokens_original)}")
print(f"Total tokens sin stopwords: {len(all_tokens_clean)}")
print(f"Reducción: {((len(all_tokens_original) - len(all_tokens_clean)) / len(all_tokens_original) * 100):.1f}%")

# Mostrar las palabras más comunes antes de eliminar stopwords
print("\nTop 10 palabras más comunes (incluyendo stopwords):")
for word, count in freq_original.most_common(10):
    print(f"  {word}: {count}")

print("\nTop 10 palabras más comunes (sin stopwords):")
for word, count in freq_clean.most_common(10):
    print(f"  {word}: {count}")

# %%
# Preparar datos para la siguiente fase
print("\n" + "=" * 80)
print("PREPARACIÓN PARA LA FASE 3: ANÁLISIS")
print("=" * 80)

# Crear dataset optimizado para análisis
df_for_analysis = df_processed[[
    'titulo', 'texto_original', 'texto_limpio', 'texto_sin_stopwords',
    'texto_stemmed', 'puntuacion', 'fecha', 'usuario',
    'num_palabras_original', 'num_palabras_sin_stopwords', 'densidad_lexica'
]].copy()

# Guardar dataset para análisis
df_for_analysis.to_csv('../data/processed/reviews_for_analysis.csv', index=False, encoding='utf-8')

print(f"Dataset para análisis guardado: ../data/processed/reviews_for_analysis.csv")
print(f"Número de registros: {len(df_for_analysis)}")
print(f"Columnas: {list(df_for_analysis.columns)}")

# %%
# Resumen del preprocesamiento
print("\n" + "=" * 80)
print("RESUMEN DEL PREPROCESAMIENTO")
print("=" * 80)
print(f"✓ Textos procesados: {len(df_processed)}")
print(f"✓ Stopwords eliminadas: {len(preprocessor.stop_words)} palabras")
print(f"✓ Reducción promedio de palabras por texto: {((df_processed['num_palabras_original'] - df_processed['num_palabras_sin_stopwords']).mean()):.1f}")
print(f"✓ Densidad léxica promedio: {df_processed['densidad_lexica'].mean():.2%}")
print(f"✓ Archivos generados:")
print(f"   - reviews_preprocessed.csv (dataset completo)")
print(f"   - reviews_for_analysis.csv (dataset optimizado)")
print(f"   - metricas_preprocesamiento.png (visualizaciones)")
print("\nEl dataset está listo para el análisis de sentimiento y frecuencia.")